# Motif search and HMMs

In this labs, we will introduce BioPython's facilities to create and search for motifs including building HMMs.

## Sequence motifs

Routines for dealing with motifs are implemented in the [Bio.motifs](https://biopython.org/docs/latest/api/Bio.motifs.html) package. It contains classes to interface with several motif databases such trasncription factors databases [TRANSFAC](https://genexplain.com/transfac/) or [JASPAR](https://jaspar.genereg.net/) using the [Bio.motifs.transfac](https://biopython.org/docs/latest/api/Bio.motifs.transfac.html) and [Bio.motifs.jaspar](https://biopython.org/docs/latest/api/Bio.motifs.jaspar.html) modules, or motif generation tool [MEME](https://meme-suite.org/meme/) using the [Bio.motifs.meme](https://biopython.org/docs/latest/api/Bio.motifs.meme.html) module. 

Here, we will show how to utilize BioPython directly to obtain a motif. The main class here is [Bio.motifs](https://biopython.org/docs/latest/api/Bio.motifs.html) which can be used to create a motif from a set of [Bio.Seq](https://biopython.org/docs/latest/api/Bio.Seq.html) objects. Let's create a DNA motif.

In [1]:
from Bio import motifs
from Bio.Seq import Seq

dna_motif = motifs.create([
    Seq("TACAA"),
    Seq("TACGC"),
    Seq("TACAC"),
    Seq("TACCC"),
    Seq("AACCC"),
    Seq("AATGC"),
    Seq("AATGC")
    ])
print("The motif consists of the following {} sequences:".format(len(dna_motif)))
print(dna_motif)

The motif consists of the following 5 sequences:
TACAA
TACGC
TACAC
TACCC
AACCC
AATGC
AATGC



The motif package is actually more motivated by DNA motifs which is evidenced by the available parsers as well by the fact that the default alphabet to be used with motifs is the nucleotide one. In order to create a protein motif, one needs to specify the alphabet to be used (notice, that the [Bio.Alphabet](https://biopython.org/docs/latest/api/Bio.Alphabet.html) package has been retired from BioPython and thus the allowed characters need to be explicitely enumerated). Let's use the coronavirus spike proteins PFAM family from the previous labs.

In [3]:
from Bio import SeqIO 
spike_proteins = list(SeqIO.parse("data/PF01600_serialized.faa", "fasta"))
pt_alphabet='ACDEFGHIKLMNPQRSTVWY-'
pt_motif = motifs.create([p.seq for p in spike_proteins], alphabet=pt_alphabet)

In [4]:
print(pt_motif)

----------------------YQVL-P---DSGEFSDNLFTVGDDGSIPP-SFGFNNWFVLSNSSSIISGTVVSNQPLRLT---C--LWPIP-----SSTGALATI---YFNGTN----GA-QCN------------GFDS--NAPFDAIRFNL--NGTLSGHNFVS----GFVLHAANGATLGFSCTNSTDAPYLR-------QIPFGI-GDT-PYYCYLNV---------TTDINSTMSFVGALPLNLREIVIA-SNGDVYMNGYRYFAAGDLSSVDVELPSQQV--FGSTFWTIAFTVFETVLLEVDGTSINRMLYCD--NPL-NRVKCSHTQFDLVDGFYPLT--DVDLAVKPFTF-VTLPTFADHSFVYFNFSLMF-----DDLN----------EDFRLQSFNLTINGQL---------SYCVQSRQFTT-SGSVRTNT------------------NHQFGFYTQRAAS---------NGCPFTIDTLNNYLTFGRICFSFG-ESGAGCGVDVMVESQYNMFKVT---T---IFVSYSEGDIIAGMPK-
-------------MRANIRNSQ--------------TDVCTTIQQGGFIPS-TFTFPQWYVLTNGSTFLQGEYTLSQPLLAN---A--HFCPR-----KNSDGYWRY---SFNNSCL-FPDH-RCQDHWYDSQNPICLGWNNT-FGLSDNIRINI--NISHDEYQSHG---GYVSLTLESGSVVNITCTNNSDPSTVTL---ATSLLPWARAIDQ-PMYCFANL---------TTGTASQLDFMGMLPPLVSELAFD-RTGGIYINGYRYYLTSALRDVDFKLKRND----TAEYFAVTWANYTDVHLSVDAGAIEKIKYCN--TPL-DRLACDMNVFNLSDGVYSYT--SLEKASVPETF-VTLPVYSNHTYVTINTSYTV-----GSCV------NCPPISS-----TIDIMHARND-------TLCVNSRQFTV-RLNT

The original sequence are stored in `motif.instances`, but what is more intersting is that by creating a motif, a consensus sequence and count matrix is created.

In [4]:
print("DNA motif consensus: {}\nProtein motif consensus: {}".format(dna_motif.consensus, pt_motif.consensus))

DNA motif consensus: TACGC
Protein motif consensus: ----------------------Y--C-D---NCAGFPANVFAVQEGGYIPP-DFSFNNWFLLTNSSTPVSGRFVSNQPLLLN---C--LWPVP-----SLTGNALPV---YFNGSG----NA-QCN------------GASN--NGTVDAIRFNL--NFTDSV-S-KG----VISLNTTGGV-YNFSCTNSSTPTTAS-------VIPFGV-TDQ-PYYCFVNY----------T-NETTLKFLGILPPSVREIVIS-RYGDFYINGYRYFSTGPLDSVSFNLTTGD----SSDFWTVAFANYTEVLVEVNNTAIQNILYCN--SPV-NRIKCQQLTFNLDDGFYSVS--SIEVGELPRTF-VTLPKFVTHSFVNITVGVSF-----D-SG-------GPPIAS-----TLTINGDN-D-------TVCVDTRQFTV-YLNVTCF---------------------DSYDATAVIQT---------GTCPFSFDKLNNYLTFGSICFSLS-P-GGGCTMDVV--TGWNGQVVK---S---LYVSYTEGDNITGVPK-


The motif object also contains the `count` matrix which counts all the occurence of all the characters in the motif.

In [5]:
print(dna_motif.counts)
for symbol in dna_motif.alphabet:
    print("Counts of {}: {}".format(symbol, dna_motif.counts[symbol]))


        0      1      2      3      4
A:   3.00   7.00   0.00   2.00   1.00
C:   0.00   0.00   5.00   2.00   6.00
G:   0.00   0.00   0.00   3.00   0.00
T:   4.00   0.00   2.00   0.00   0.00

Counts of A: [3, 7, 0, 2, 1]
Counts of C: [0, 0, 5, 2, 6]
Counts of G: [0, 0, 0, 3, 0]
Counts of T: [4, 0, 2, 0, 0]


For practical use, more interesting is the availability of the [position weigh matrix (PWM)](https://biopython.org/docs/1.75/api/Bio.motifs.matrix.html?highlight=log_odds#Bio.motifs.matrix.PositionWeightMatrix.log_odds) and [position specific scoring matrix (PSSM)](https://biopython.org/docs/1.75/api/Bio.motifs.matrix.html?highlight=log_odds#Bio.motifs.matrix.PositionSpecificScoringMatrix). PWM is just the normalized count matrix (contains frequencies).

In [7]:
print(dna_motif.pwm)
print(dna_motif.pssm)

        0      1      2      3      4
A:   0.43   1.00   0.00   0.29   0.14
C:   0.00   0.00   0.71   0.29   0.86
G:   0.00   0.00   0.00   0.43   0.00
T:   0.57   0.00   0.29   0.00   0.00

        0      1      2      3      4
A:   0.78   2.00   -inf   0.19  -0.81
C:   -inf   -inf   1.51   0.19   1.78
G:   -inf   -inf   -inf   0.78   -inf
T:   1.19   -inf   0.19   -inf   -inf



As seen above, neither PWM nor PSSM contain pseudocounts. To get PWM and PSSM with pseudocounts, we can use the count matrix and compute them by ourselves.

In [9]:
# pwm = dna_motif.counts.normalize(pseudocounts=0.5)
pwm = dna_motif.counts.normalize(pseudocounts={'A':0.6, 'C': 0.4, 'G': 0.4, 'T': 0.6})
print(pwm)

        0      1      2      3      4
A:   0.40   0.84   0.07   0.29   0.18
C:   0.04   0.04   0.60   0.27   0.71
G:   0.04   0.04   0.04   0.38   0.04
T:   0.51   0.07   0.29   0.07   0.07



To obtain the PSSM, we can use the [log_ods](https://biopython.org/docs/1.75/api/Bio.motifs.matrix.html?highlight=log_odds#Bio.motifs.matrix.PositionWeightMatrix.log_odds) method of the `PositionWeightMatrix`. It computes log odds based on the frequencies and background distribution (uniform, by default).

In [10]:
pssm = pwm.log_odds()
print(pssm)

        0      1      2      3      4
A:   0.68   1.76  -1.91   0.21  -0.49
C:  -2.49  -2.49   1.26   0.09   1.51
G:  -2.49  -2.49  -2.49   0.60  -2.49
T:   1.03  -1.91   0.21  -1.91  -1.91



Using a motif, we can search for exact matches in a sequence.

In [11]:
test_seq=Seq("TACACTGCATTACAACCCAAGCATTA")

In [12]:
for pos, seq in dna_motif.instances.search(test_seq):
    print("{} {}".format(pos, seq))

0 TACAC
10 TACAA
13 AACCC


Hower, having PSSM enables also to search for motifs which are more probable than background (score > 0) or specified threshold.

In [ ]:
for position, score in pssm.search(test_seq, threshold=3.0):
    print("Position {}: score = {:.3f}".format(position, score))

Position 0: score = 5.768
Position -20: score = 4.746
Position 10: score = 3.768
Position 13: score = 5.298
Position -6: score = 4.746


If we are interested in scorse at all the positions we can do that, too.

In [20]:
pssm.calculate(test_seq)

array([  5.7675514 ,  -5.534453  ,  -3.8714879 ,   0.49856207,
        -7.2893405 ,  -1.8954138 , -10.704378  ,  -2.9259357 ,
         0.69650143,  -3.180816  ,   3.7675512 ,  -2.0039382 ,
        -1.041413  ,   5.298437  ,  -0.9494905 ,  -4.003938  ,
        -9.173863  ,  -0.53891265,  -0.45645046,  -2.2490509 ,
       -10.704378  ,  -2.9259357 ], dtype=float32)

The motif object includes an interface to the [WebLogo](http://weblogo.threeplusone.com/) tool. Be aware that it is indeed just an API call.

In [12]:
dna_motif.weblogo("dna_logo.png")
pt_motif.weblogo("spike_protein_logo.png",  alphabet='alphabet_protein', sequence_type='protein')

### ---- Begin Exercise ----
If you try to compute PSSM for a protein motiv (`pssm.calculate(pt_seq)`), you get an error message along the following lines:
```
C:\Python39\lib\site-packages\Bio\motifs\matrix.py in calculate(self, sequence)
    340         # TODO - Code itself tolerates ambiguous bases (as NaN).
    341         if sorted(self.alphabet) != ["A", "C", "G", "T"]:
--> 342             raise ValueError(
    343                 "PSSM has wrong alphabet: %s - Use only with DNA motifs" % self.alphabet
    344             )
```
This means that PSSM is actually implemented only for nucleotide sequences in the current (1.75) version of BioPython.
 
Implement code to carry out the `pwm.calculate` method for protein sequences. The method can be implemented either as an individual function or by extending the respective implementations, for example by adding `calculate_proteins` into the PSM class.
### ---- End Exercise ----

## HMMs

BioPython supports hidden markov models via the [Bio.HMM](https://biopython.org/docs/1.74/api/Bio.HMM.MarkovModel.html) model. Specifically, it implements building and usage of HMM in the [Bio.HMM.MarkovModel](https://biopython.org/docs/1.74/api/Bio.HMM.MarkovModel.html) module and training in the [Bio.HMM.Trainer](https://biopython.org/docs/1.74/api/Bio.HMM.Trainer.html) module. The trainer includes both supervised and unsupervised (Baum-Welch algorithm) training.

In the following example, we will implement the dishonest casino example (see the lecture).

First, we create the emission probabilities for the fair and loaded dice.

In [14]:
import numpy as np
from random import choices

fair_weights = list(np.ones(6) * 1/6)
loaded_weights = [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]

Next, we load the required package and create the model using `MarkovModelBuilder`. The builder is simply passed all the model defining transition and emission probabilities.

In [16]:
from Bio.HMM import MarkovModel
from Bio.HMM.Utilities import pretty_print_prediction

In [17]:
states = {'fair': 'F', 'loaded': 'L'}
alphabet = list(range(1,7))
mm_builder = MarkovModel.MarkovModelBuilder(state_alphabet=[states['fair'], states['loaded']], emission_alphabet=alphabet)
mm_builder.set_initial_probabilities({states['fair']: 0.5, states['loaded']: 0.5})
mm_builder.allow_transition(states['fair'], states['fair'], 0.95)
mm_builder.allow_transition(states['fair'], states['loaded'], 0.05)
mm_builder.allow_transition(states['loaded'], states['loaded'], 0.9)
mm_builder.allow_transition(states['loaded'], states['fair'], 0.1)
for symbol in alphabet:
    mm_builder.set_emission_score(states['fair'], symbol, fair_weights[symbol-1])
    mm_builder.set_emission_score(states['loaded'], symbol, loaded_weights[symbol-1])
    
m = mm_builder.get_markov_model()

Let's apply the model on a simple example where the rolls are generated eigher from the fair or loaded states.

In [18]:
rolls=choices(population=list(range(1,7)), weights= fair_weights, k=10)
path = m.viterbi(rolls, states.values())
print(path)
rolls=choices(population=list(range(1,7)), weights= loaded_weights, k=10)
path = m.viterbi(rolls, states.values())
print(path)

(Seq('FFFFFFFFFF'), -19.072381522328445)
(Seq('LLLLLLLLLL'), -13.401177364382136)


Now lets try to generate a sequence of rolls based on the given probabilities and let the model label it. 

In [19]:
s = choices(population=list(['F', 'L']), weights=[0.5,0.5], k=1)[0]
t= {
    'F': [0.95, 0.05],
    'L': [0.1, 0.9]    
}
e={
    'F': fair_weights,
    'L': loaded_weights
}
rolls = []
true_states = []
for i in range(200):
    true_states.append(s)
    rolls.append(choices(population=list(range(1,7)), weights=e[s], k=1)[0])
    s = choices(population=list(['F', 'L']), weights=t[s], k=1)[0]
print(rolls)
print(true_states) 
    

[4, 2, 2, 1, 1, 3, 2, 1, 3, 6, 1, 6, 6, 6, 5, 5, 3, 6, 6, 5, 6, 5, 6, 6, 6, 6, 6, 5, 6, 3, 6, 6, 5, 6, 4, 1, 3, 5, 3, 3, 1, 1, 5, 5, 4, 6, 3, 5, 1, 2, 6, 1, 6, 1, 2, 6, 4, 3, 2, 1, 6, 5, 5, 6, 6, 4, 4, 4, 2, 6, 2, 1, 5, 1, 4, 3, 5, 1, 5, 6, 2, 5, 6, 6, 2, 1, 3, 5, 3, 1, 6, 5, 2, 2, 4, 2, 6, 4, 3, 5, 4, 6, 1, 3, 1, 2, 2, 6, 1, 1, 4, 5, 4, 5, 5, 6, 6, 6, 1, 2, 3, 4, 4, 4, 5, 2, 1, 4, 4, 6, 6, 3, 1, 2, 3, 6, 6, 2, 5, 4, 6, 4, 1, 5, 1, 5, 2, 3, 6, 2, 6, 3, 3, 2, 1, 1, 3, 6, 6, 1, 4, 2, 2, 3, 2, 3, 5, 2, 5, 6, 4, 3, 2, 4, 6, 4, 1, 1, 3, 5, 2, 6, 4, 5, 1, 6, 6, 3, 3, 2, 3, 6, 3, 6, 6, 5, 4, 1, 2, 2]
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'

In [20]:
path = m.viterbi(rolls, states.values())
print(path)

(Seq('FFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFF...FFF'), -362.76538469275863)


In [21]:
pretty_print_prediction([str(r) for r in rolls], true_states, [s for s in str(path[0])], line_width=40)

Emissions       ['4', '2', '2', '1', '1', '3', '2', '1', '3', '6', '1', '6', '6', '6', '5', '5', '3', '6', '6', '5', '6', '5', '6', '6']
Real State      ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']
Predicted State ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']

Emissions       ['6', '6', '6', '5', '6', '3', '6', '6', '5', '6', '4', '1', '3', '5', '3', '3', '1', '1', '5', '5', '4', '6', '3', '5']
Real State      ['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
Predicted State ['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']

Emissions       ['1', '2', '6', '1', '6', '1', '2', '6', '4', '3', '2', '1', '6', '5', '5', '6', '6', '4', '4', '4', '2', '6', '2', '1']
Real State      ['F', 'F', 'F', 'F', 'F

### ---- Begin Exercise ----
**This excercise is optional, but if you hand it in you earn extra 5 points for the exam.**

Implement Profile HMM for the spike protein family's MSA (`data/PF01600_serialized.faa`) based on the description in the lecture and show alignment with the SARS-CoV2 spike protein ("`data/YP_009724390.1_spike_protein.fa`)
### ---- End Exercise ----